# Machine Learning Demonstrations - 1

To run this notebook, you need to install python dependencies:

Install tensorflow:

With conda:

```conda install -c anaconda tensorflow-gpu```

With pip:

```pip install tensorflow-gpu```

Other dependencies:

With pip:

```pip install Keras==2.0.9
pip install scikit_learn==0.19.1```

## Classification using Neural Networks

Neural Networks are machine learning models based (loosely) on biological neural networks in the brain. Data is passed through a series of layers of *artificial neurons* , giving increasingly useful representations. Since the number of  layers can be quite large, this type of machine learning is often called *deep learning*.

Neural networks can
- Work on data with little-or-no feature engineering applied
- Be applied to *perceptive* problems (such as computer vision and voice recognition)

We will implement and manipulate Neural Networks using **Keras**, a high-level API that runs on top of a number of common deep learning frameworks, including TensorFlow, Theano and CNTK.

Further discussion of Neural Networks can be found on the [Wikipedia article](https://en.wikipedia.org/wiki/Artificial_neural_network). More information about the Keras framework can be found on the [Keras website](https://keras.io/).

## MNIST (handwritten digits) database

The Modified National Institute of Standards and Technology (MNIST) database, is a large collection of handwritten digits.

MNIST is commonly used for training and testing image processing systems, particularly in the field of machine learning.

<img src="../images/mnist.png" />

### Load data

In [ ]:
/ Utility functions
\l ../utils/funcs.q
\l ../utils/graphics.q

In [ ]:
mnist:.p.import[`keras.datasets.mnist][`:load_data;<][]
data:raze(`xtrain`ytrain;`xtest`ytest)!'mnist
@[`data;`ytrain`ytest;"f"$];
/ Inspect data
-1 string[count data`xtrain]," samples of shape ",sv[" x "]string 1_shape data`xtrain;
-1"";show 5#data`xtrain;-1"";
-1"Distribution of target values is:\n";
show update pcnt:round[;.01]100*num%sum num from select num:count i by target from([]target:data`ytrain);
-1"\nTraining/test split is:\n";
show count each data;

### Inspect a random digit

In [ ]:
\c 2000 2000
j:rand count data`xtest; / pick a number
-1"Inpecting data point ",string j;-1"";
show data[`xtest]j;-1"";
-1"Length: ",string count data[`xtest]j;
\c 16 80

In [ ]:
@[`data;`xtrain`xtest;%;255];
/ Display using matplotlib
plt[`:imshow]"f"$data[`xtest]j;
plt[`:show][];

### Inspect a random sample

In [ ]:
subplots:plt[`:subplots][5;5]
fig:subplots[@;0]
axarr:subplots[@;1]
fig[`:set_size_inches;18.5;10.5];

{ i:rand count data`xtest;
  //box:axarr[@;x 0][@;x 1];
  //box:axarr[@;tuple x];
  box:axarr[`$":__getitem__"].p.eval","sv string x;
  box[`:imshow]"f"$data[`xtest]i;
  box[`:axis]`off;
  box[`:set_title]"Label is ",string data[`ytest]i;
 }each cross[til 5;til 5];

plt[`:show][];

### Build model

In [ ]:
/ Prepare data for NN model
@[`data;`xtrain`xtest;{nparray[(raze/)x][`:reshape]count[x],28*28}];
@[`data;`ytrain`ytest;{nparray onehot x}];

In [ ]:
/ Layer used in model
dense:.p.import[`keras.layers]`:Dense;

In [ ]:
/ Build the model
model:.p.import[`keras.models;`:Sequential][];
model[`:add][dense[512;`activation pykw`relu;`input_shape pykw enlist 28*28]];
model[`:add]dense[10;`activation pykw`softmax];
model[`:compile][`loss pykw`categorical_crossentropy;`optimizer pykw`rmsprop;`metrics pykw pylist enlist`accuracy];
model[`:summary][];

### Fit model

In [ ]:
epochs:20
batchsz:128
start:.z.T
cresults:model[`:fit][data`xtrain;data`ytrain;`batch_size pykw batchsz;`verbose pykw 2;`epochs pykw epochs]
-1"Training continuous model for ",string[epochs]," epochs took ",string .z.T-start;

### Predicting

In [ ]:
preds:first each idesc each model[`:predict][data`xtest]`;
class:first each where each 1=data[`ytest]`;
show res:update Hit:Class=Prediction from([]Class:class;Prediction:preds)

### Analysing accuracy

In [ ]:
tot:select avg Hit by`$string Class from res;
show update Miss:1-Hit from tot upsert update Class:`TOTAL from select avg Hit from res;

In [ ]:
ctab:update pcnt:100*num%sum num by Class from select num:count i by Class,Prediction from res
ctab:0^(asc flip`Class`Prediction!flip cross[exec distinct Class from ctab;exec distinct Prediction from ctab])#ctab
ctab:0!update p:{`$"Pred_",string x}each Prediction from ctab
show cpivot:exec(p!num)by Class:Class from ctab

### Visualising Accuracy (3D plot)

In [ ]:
// Import matplotlib and library that allows 3D plots
.p.import[`mpl_toolkits.mplot3d]`:Axes3D;

// Initialize plot
fig:plt[`:figure][`figsize pykw 10 7];
ax:fig[`:add_subplot][111;`projection pykw"3d"];

// Definition of 3D barplots
n:count ctab;
ax[`:bar3d][ctab.Class;ctab.Prediction;n#0;n#0.5;n#0.5;ctab.num;`color pykw "lightgreen"];
ax[`:set_xlabel]["Actual labels"];
ax[`:set_ylabel]["Prediction"];
ax[`:set_zlabel]["Count"];
plt[`:show][];

### Visualising Accuracy (Heatmap)

In [ ]:
Norm_conf:flip value flip value exec(p!pcnt)by Class from ctab
fig:plt[`:figure][`figsize pykw 9 7];
ax:fig[`:add_subplot][111];
ax[`:set_aspect][1];

orig_cmap:plt[`:get_cmap][];
plt[`:set_cmap]"jet";
resIm:ax[`:imshow][Norm_conf;`interpolation pykw`nearest];
ax[`:imshow][Norm_conf;`interpolation pykw`nearest];
ax[`:annotate][;;`horizontalalignment pykw`center; `verticalalignment pykw`center]'[string ctab`num;flip ctab`Prediction`Class];

// Customize plot
fig[`:colorbar]resIm;
plt[`:xticks]til 10;
plt[`:yticks]til 10;
plt[`:show][];
plt[`:set_cmap]orig_cmap;

### Where are mistakes made?

In [ ]:
ind:til count cmat:flip value flip value cpivot
mclass:(corr;(sum each cmat)-corr;(sum each flip cmat)-corr:cmat ./:2#'til count cmat)
width:0.3
// We initialize the plot
ax:plt[`:subplots][`figsize pykw 10 8][@;1]
// Adding the bars related to the correct classifications (in green)
ax[`:bar][ind;mclass 0;width;`color pykw"g";`label pykw"Correctly classified"];
ax[`:bar][ind+width;mclass 1;width;`color pykw"r";`label pykw"This misclassified as others"];
ax[`:bar][ind-width;mclass 2;width;`color pykw"b";`label pykw"Others misclassified as this"];

ax[`:set_ylabel]"Count";
ax[`:set_title]"NN results";
ax[`:set_xticks]ind;
ax[`:set_xticklabels]ind;
ax[`:legend][`loc pykw"center left";`shadow pykw 1b;`bbox_to_anchor pykw(1,0.5);`fontsize pykw 15];
plt[`:show][];

### Check mistakes

In [ ]:
mistakes:16?exec i from res where not Hit

subplots:plt[`:subplots][4;4]
fig:subplots[@;0]
axarr:subplots[@;1]
fig[`:set_size_inches][18.5;10.5];

{[x;i]
  box:axarr[`$":__getitem__"].p.eval","sv string x;
  box[`:imshow]28 28#255*data[`xtest][`]i;
  box[`:axis]`off;
  box[`:set_title]["Prediction is ",string res[i;`Prediction]];
 }'[cross[til 4;til 4];mistakes];

plt[`:show][];